# Computation Method

The SEMNAN Solver has two "equivalent" computational methods.
These two methods result in the same weight partial derivative in each step.
However, you might computationally prefer one method above the other.

The two computational methods of the SEMNAN Solver are:
- the covariance method, and
- the weight accumulation method.
By default, the SEMNAN Solver uses the covariance method. However, one might use the accumulation method at will.

Let's get started with the same example as in [introduction](introduction.ipynb).

In [1]:
import torch
import semnan_cuda as sc

device = torch.device("cuda")

struct = torch.tensor([
        [1, 1, 1, 0],
        [0, 1, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 1, 1],  # V_X
        [0, 0, 1, 0],  # V_BP
        [0, 0, 0, 1],  # V_BMI
        [0, 0, 0, 0],  # V_Y
    ], dtype=torch.bool, device=device)

sample_covariance = torch.tensor([
        [2,  3,  6,  8],
        [3,  7, 12, 16],
        [6, 12, 23, 30],
        [8, 16, 30, 41],
    ], device=device)

We may initialize the solver with the accumulation method, instead of the usual method.

In [2]:
semnan = sc.SEMNANSolver(
    struct, 
    sample_covariance=sample_covariance, 
    method=sc.SEMNANSolver.METHODS.ACCUM
)

Note that we set `method=sc.SEMNANSolver.METHODS.ACCUM` in the initialization. Alternatively, we could use `method=sc.SEMNANSolver.METHODS.COVAR`.
Now, we can start the computation using the same algorithm as in [introduction](introduction.ipynb).

In [3]:
max_iterations = 10000
min_error = 1.0e-7
optim = torch.optim.Adamax([semnan.weights], lr=0.001)

for i in range(max_iterations):
    semnan.forward()
    error = semnan.loss().item()

    if error < min_error:
        break

    semnan.backward()
    optim.step()

    if i % (max_iterations / 10) == 0:
        print(f"iteration={i:<10} loss={error:<15.5}")
else:
    print("Did not converge in the maximum number of iterations!")

iteration=0          loss=1.6577         
iteration=1000       loss=0.085377       
iteration=2000       loss=0.00069857     


The resulting visible covariance matrix is:

In [4]:
print(semnan.visible_covariance_)

tensor([[ 1.9997,  2.9992,  5.9987,  7.9978],
        [ 2.9992,  6.9994, 11.9984, 15.9974],
        [ 5.9987, 11.9984, 22.9963, 29.9953],
        [ 7.9978, 15.9974, 29.9953, 40.9930]], device='cuda:0')
